<a href="https://colab.research.google.com/github/nedokormysh/GB_PyTorch/blob/lesson9/GB_PyTorch_hw_9_dirty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание

1. Возьмите готовую модель из https://huggingface.co/models для классификации сентимента текста.
2. Сделайте предсказания на всем df_val. Посчитайте метрику качества.
3. Дообучите эту модель на df_train. Посчитайте метрику качества на df_val.

Данные на google drive: https://drive.google.com/file/d/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 18.4 MB/s eta 0:00:00


In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter
from transformers import pipeline
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, BertModel

import pandas as pd

Загрузка датасета.

In [3]:
!wget 'https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901' -O data.zip

--2023-01-09 09:23:00--  https://drive.google.com/uc?export=download&id=1Mev_EEput0LlBj8MDHIJkBtahlJ6J901
Resolving drive.google.com (drive.google.com)... 108.177.126.113, 108.177.126.100, 108.177.126.101, ...
Connecting to drive.google.com (drive.google.com)|108.177.126.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ssgtthrff0i6plbuo701587o2aha590q/1673256150000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=0acd9e55-696c-4959-9d91-7e61054f1c96 [following]
--2023-01-09 09:23:04--  https://doc-14-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ssgtthrff0i6plbuo701587o2aha590q/1673256150000/14904333240138417226/*/1Mev_EEput0LlBj8MDHIJkBtahlJ6J901?e=download&uuid=0acd9e55-696c-4959-9d91-7e61054f1c96
Resolving doc-14-c0-docs.googleusercontent.com (doc-14-c0-docs.googleusercontent.com)... 108.177.119.132, 

In [4]:
!unzip data.zip

Archive:  data.zip
  inflating: train.csv               
  inflating: val.csv                 


In [5]:
df_train = pd.read_csv("train.csv")
df_val = pd.read_csv("val.csv")

df_train.shape, df_val.shape

((181467, 3), (22683, 3))

In [6]:
df_train.head()

,id,text,class
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1


In [7]:
df_val.head()

,id,text,class
0,181467,RT @TukvaSociopat: Максимальный репост! ))) #є...,1
1,181468,чтоб у меня з.п. ежегодно индексировали на инд...,0
2,181469,@chilyandlime нехуя мне не хорошо !!! :((((,0
3,181470,"@inafish нее , когда ногами ахахах когда?ахаха...",0
4,181471,"Хочу сделать как лучше, а получаю как всегда. :(",0


In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Den4ikAI

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("Den4ikAI/russian_sensitive_topics")
model_Den4= AutoModelForSequenceClassification.from_pretrained("Den4ikAI/russian_sensitive_topics")

Downloading:   0%|          | 0.00/36.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

In [ ]:
# print(model_Den4)
print("Parameters full train:", sum([param.nelement() for param in model_Den4.parameters()]))

Parameters full train: 427311497


In [ ]:
sentDen4 = pipeline('text-classification', model='Den4ikAI/russian_sensitive_topics')
sentDen4("Я убью тебя с ак47")

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

[{'label': 'offline_crime,weapons', 'score': 0.7298686504364014}]

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('Den4ikAI/russian_sensitive_topics')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

tensor([[  101,  2496, 14637,   849, 21298,   679,  1693,   102,     0,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [ ]:
tokenizer.ids_to_tokens[14637], tokenizer.ids_to_tokens[21298]

('текста', 'ток')

In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] пример текста для токенизации [SEP] [PAD] [PAD]


In [ ]:
sentDen4 = pipeline('text-classification', model='Den4ikAI/russian_sensitive_topics')

idx = 3
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
print('label by model is', sentDen4(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentDen4(df_train.iloc[idx]['text'])[0]['score'])

RT @epupybobv: Хочется котлету по-киевски. Запретный плод. :)
label is 1
label by model is none with score 0.5054712295532227


Препроцессинг

In [ ]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('Den4ikAI/russian_sensitive_topics')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [ ]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [ ]:
# for txt, lbl in train_loader:
#     print(txt.keys()) #словарь с ключами'input_ids', 'token_type_ids', 'attention_mask'
#     print(txt['input_ids'].shape) #тензор размера (B,1,max_len) из id токенов
#     print(txt['attention_mask'].shape) #тензор размера (B,1,max_len) из индексов, указывающих, на какие токеты модель должна обратить внимание
#     break

In [ ]:
from transformers import AutoModelForSequenceClassification

model_Den4= AutoModelForSequenceClassification.from_pretrained("Den4ikAI/russian_sensitive_topics")
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = model_Den4.from_pretrained("Den4ikAI/russian_sensitive_topics")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

## Skolkovo

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

model_bert = BertForSequenceClassification.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')
# print(model_bert)
print("Parameters full train:", sum([param.nelement() for param in model_bert.parameters()]))

In [ ]:
# Пример классификации

sentSkolkovo = pipeline("text-classification", model='SkolkovoInstitute/russian_toxicity_classifier')
sentSkolkovo("Я убью тебя с ак47")

In [ ]:
# Пример токенизации

from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('SkolkovoInstitute/russian_toxicity_classifier')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

In [ ]:
tokenizer.ids_to_tokens[17863], tokenizer.ids_to_tokens[15427]

In [ ]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

In [ ]:
sentSkolkovo = pipeline('text-classification', model='Den4ikAI/russian_sensitive_topics')

idx = 3
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
print('label by model is', sentSkolkovo(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentSkolkovo(df_train.iloc[idx]['text'])[0]['score'])

## RuBert-toxic

Возьмём модель sismetanin/rubert-toxic-pikabu-2ch.

In [9]:
# Загрузка модели. Проверка работы модели на токсичной фразе
sentRuBert = pipeline('text-classification', model='sismetanin/rubert-toxic-pikabu-2ch')
sentRuBert("Я убью тебя с ак47")

Downloading:   0%|          | 0.00/920 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': 'LABEL_1', 'score': 0.8942258358001709}]

In [28]:
# # Проверка работы модели на фразе с оскорблением.
# model = BertForSequenceClassification.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
# batch = tokenizer.encode('Я убью тебя с ак47', return_tensors='pt')
# model(batch)

In [10]:
# Загрузка токенайзера
tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])
print(sentRuBert(example_text))

tensor([[   101,  22632,  22574,   2748, 111601,  17510,   3426,    102,      0,
              0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])
[{'label': 'LABEL_0', 'score': 0.9928885102272034}]


In [11]:
tokenizer.ids_to_tokens[22632], tokenizer.ids_to_tokens[111601]

('пример', 'токе')

In [12]:
example_text = tokenizer.decode(bert_input.input_ids[0])

print(example_text)

[CLS] пример текста для токенизации [SEP] [PAD] [PAD]


In [13]:
# Проверка работы модели на примере из датасета.

idx = 5
print(df_train.iloc[idx]['text'])
print('label is', df_train.iloc[idx]['class'])
# print('label by model is', sentRuBert(df_train.iloc[idx]['text'])[0]['label'], 'with score', sentRuBert(df_train.iloc[idx]['text'])[0]['score'])
print(f'label by model is {sentRuBert(df_train.iloc[idx]["text"])[0]["label"]} with score {round(sentRuBert(df_train.iloc[idx]["text"])[0]["score"], 5)}')

Манчестер через час играет, а я не дома (
label is 0
label by model is LABEL_0 with score 0.99585


Препроцессинг.

In [14]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [16]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=256,
                          shuffle=True,
                          num_workers=256)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 256 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [17]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([256, 1, 10])


In [18]:
# from torch import nn
# from transformers import BertModel


class BertClass(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        # self.bert = model_RuBert.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self.bert = BertModel.from_pretrained('sismetanin/rubert-toxic-pikabu-2ch')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        # print()
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

### Предсказание на валидационной выборке

In [19]:
model = BertClass().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at sismetanin/rubert-toxic-pikabu-2ch were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
# print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

Parameters full train: 177854978
Parameters transfer learning: 1538


In [66]:
# # Списки для истинного и спрогнозированного класса объектов.
# list_y_true = []
# list_y_pred = []

# # Прогноз на валидационной выборке.
# for x, y in tqdm(df_val):
#     y_pred = model.forward(x)
    
#     list_y_true += [*y.numpy()]
#     list_y_pred += [*y_pred.numpy()]

In [67]:
# # Списки для истинного и спрогнозированного класса объектов.
# list_y_true = []
# list_y_pred = []

# # Прогноз на валидационной выборке.
# for x, y in tqdm(df_val):
#     # print(y.shape)
#     # y_pred = model.forward(x)
    
#     # list_y_true += [*y.numpy()]
#     # list_y_pred += [*y_pred.numpy()]

In [65]:
# a = [1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#         1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
#         0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1]
# len(a)

64

In [127]:
# len(df_val['text'])

In [120]:
%%time
model.eval()

list_y_true = []
list_y_pred = []

test_acc, test_tp, test_fp, test_tn, test_fn = 0, 0, 0, 0, 0

# for j, data in enumerate(valid_loader):
#     test_labels = data[1].to(device)
#     test_outputs = model(data[0].to(device))

for val_input, val_label in valid_loader:
    # print(val_label)
    list_y_true += val_label
    val_label = val_label.to(device)
    mask = val_input['attention_mask'].to(device)
    input_id = val_input['input_ids'].squeeze(1).to(device)

    output = model(input_id, mask)

    list_y_pred += output.argmax(dim=1)

    test_acc += (output.argmax(dim=1) == val_label).sum().item()
    test_tp += ((val_label == 1) & (output.argmax(dim=1) == 1)).sum().item()
    test_tn += ((val_label == 0) & (output.argmax(dim=1) == 0)).sum().item()
    test_fp += ((val_label == 0) & (output.argmax(dim=1) == 1)).sum().item()
    test_fn += ((val_label == 1) & (output.argmax(dim=1) == 0)).sum().item()

test_precision = test_tp / (test_tp + test_fp) if (test_tp + test_fp) != 0 else 0
test_recall = test_tp / (test_tp + test_fn) if (test_tp + test_fn) != 0 else 0
test_acc = test_acc / len(df_val)


test_f1_score = 2 * test_precision * test_recall / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0

print(f'test accuracy = {test_acc: .3f}, test_precision = {test_precision: .3f}, test_recall = {test_recall: .3f}, test F1 score = {test_f1_score: .3f}')

test accuracy =  0.529, test_precision =  0.525, test_recall =  0.716, test F1 score =  0.605
CPU times: user 12.4 s, sys: 382 ms, total: 12.8 s
Wall time: 13.8 s


In [121]:
list_y_true = [list_y_true[i].item() for i in range(len(list_y_true))]
list_y_pred = [list_y_pred[i].item() for i in range(len(list_y_pred))]
print(classification_report(y_true=list_y_true, y_pred=list_y_pred))

              precision    recall  f1-score   support

           0       0.54      0.34      0.42     11234
           1       0.52      0.72      0.61     11449

    accuracy                           0.53     22683
   macro avg       0.53      0.53      0.51     22683
weighted avg       0.53      0.53      0.51     22683



In [116]:
# list_y_pred = [list_y_pred[i].item() for i in range(len(list_y_true))]

In [122]:
# list_y_pred

In [123]:
# print(list_y_true.)

In [100]:
# list_y_true[0].item()

1

In [103]:
# list_y = []
# for i in range(len(list_y_true)):
#   list_y.append(list_y_true[i].item())

In [124]:
# list_y_true = [list_y_true[i].item() for i in range(len(list_y_true))]
# list_y_true

In [92]:
# list_y = []
# for el in list_y_true:
#   # print(el.numpy())
#   list_y.append(el.numpy())

In [125]:
# list_y

In [78]:
val_label

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0,
        1, 0, 1], device='cuda:0')

In [57]:
val_label.cpu().numpy()

array([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 0, 1])

In [126]:
# list_y_true = []
# list_y_pred = []

# for val_input, val_label in valid_loader:
#         val_label = val_label.to(device)
#         mask = val_input['attention_mask'].to(device)
#         input_id = val_input['input_ids'].squeeze(1).to(device)

#         output = model(input_id, mask)

#         # list_y_true += [*y.numpy()]
#         # list_y_pred += [*y_pred.numpy()]

In [ ]:
# for train_input, train_label in tqdm(train_loader):
#         mask = train_input['attention_mask'].to(device)
#         input_id = train_input['input_ids'].squeeze(1).to(device)
#         train_label = train_label.to(device)

In [ ]:
# input_id.shape

In [ ]:
# input_id

In [ ]:
# mask.shape

### Дообучение модели

In [ ]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()

    list_y_true = []
    list_y_pred = []
    
    total_loss_val, total_acc_val = 0.0, 0.0
    test_acc, test_tp, test_fp, test_tn, test_fn = 0, 0, 0, 0, 0

    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        list_y_pred += output.argmax(dim=1)

        test_acc += (output.argmax(dim=1) == val_label).sum().item()
        test_tp += ((val_label == 1) & (output.argmax(dim=1) == 1)).sum().item()
        test_tn += ((val_label == 0) & (output.argmax(dim=1) == 0)).sum().item()
        test_fp += ((val_label == 0) & (output.argmax(dim=1) == 1)).sum().item()
        test_fn += ((val_label == 1) & (output.argmax(dim=1) == 0)).sum().item()

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
    
    test_precision = test_tp / (test_tp + test_fp) if (test_tp + test_fp) != 0 else 0
    test_recall = test_tp / (test_tp + test_fn) if (test_tp + test_fn) != 0 else 0
    test_acc = test_acc / len(df_val)

    test_f1_score = 2 * test_precision * test_recall / (test_precision + test_recall) if (test_precision + test_recall) != 0 else 0
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')
    
    print(f'test accuracy = {test_acc: .3f}, test_precision = {test_precision: .3f}, test_recall = {test_recall: .3f}, test F1 score = {test_f1_score: .3f}')

  0%|          | 0/709 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 256 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# JP

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

example_text = 'Пример текста для токенизации'
bert_input = tokenizer(example_text, padding='max_length', max_length=10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['attention_mask'])

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

tensor([[  101, 14337, 33930, 77572, 10520, 84964, 15065, 33917,   102,     0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 0]])


In [ ]:
class TwitterDataset(torch.utils.data.Dataset):
    
    def __init__(self, txts, labels):
        self._labels = labels
        
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]
        
    def __len__(self):
        return len(self._txts)
    
    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [ ]:
y_train = df_train['class'].values
y_val = df_val['class'].values

train_dataset = TwitterDataset(df_train['text'], y_train)
valid_dataset = TwitterDataset(df_val['text'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [ ]:
for txt, lbl in train_loader:
    print(txt.keys())
    print(txt['input_ids'].shape)
    break

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
torch.Size([64, 1, 10])


In [ ]:
from torch import nn
from transformers import BertModel


class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):
        
        _, pooled_output = self.bert(input_ids=x, attention_mask=mask, return_dict=False)
        # _, pooled_output - набор эмбеддинигов слов, эмбеддинг предложения
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [ ]:
model = BertClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

Parameters full train: 177854978
Parameters transfer learning: 1538


In [ ]:
for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

100%|██████████| 2836/2836 [00:24<00:00, 115.04it/s]


In [ ]:
mask.shape

torch.Size([27, 1, 10])

In [ ]:
input_id.shape

torch.Size([27, 10])

In [ ]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)
                
        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()
                
        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()
            
    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()
                    
        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc
            
    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

  0%|          | 2/2836 [00:27<10:53:44, 13.84s/it]


KeyboardInterrupt: ignored